In [ ]:
import langchain

from langchain_openai import ChatOpenAI

In [ ]:
import os

with open('OPENAI_API_KEY', 'r') as file:
    env_value = file.read().strip()

In [ ]:
os.environ['OPENAI_API_KEY'] = env_value

In [ ]:
llm = ChatOpenAI()

In [ ]:
model = ChatOpenAI(model_name="gpt-3.5")

In [ ]:
# llm.invoke("hi there!")